In [5]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 4        # 선박 수
T = 24       # 전체 시간 슬롯 (1시간 단위: t=1,...,24)
# Q는 전체 크레인 수 (여기서는 10)
Q_total = 10  

# 선박별 총 작업시간 (크레인-시간)
a = {1: 5, 2: 5, 3: 5, 4: 4}

# 입항 및 출항 시각 (t 단위)
b = {1: 5, 2: 3, 3: 11, 4: 15}
d = {1: 8, 2: 10, 3: 15, 4: 22}

# 선박별로 사용 가능한 크레인 집합 (데이터로 제공)
# 예: 선박 1은 크레인 1~3, 선박 2는 크레인 5~7, 선박 3은 크레인 1~3, 선박 4는 크레인 2~3 사용 가능
available_cranes = {
    1: [1, 2, 3],
    2: [6, 7, 8],
    3: [1, 2, 3],
    4: [4, 5]
}

# 시간당 최소/최대 크레인 할당 (여기서는 선박별로 사용가능한 크레인 수의 상한을 반영)
# 예: 선박 i에 대해, qmax[i]는 available_cranes[i]에 포함된 크레인 수의 최대값
qmin = {1: 0, 2: 0, 3: 0, 4: 0}
qmax = {i: len(available_cranes[i]) for i in available_cranes}

# 계약시간 (최대 계약시간)
H = 3

# 단가
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가

# 야간 할증 계수: 예를 들어, t=18~24와 t=1~4는 야간(1.5), 나머지는 주간(1.0)
w = {t: 1.5 if (t >= 18 or t <= 4) else 1.0 for t in range(1, T+1)}

# Big-M 값들
M = 50 
M1 = 30   # (57) 제약용 Big-M

# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP_with_AvailableCrane")

# ----------------------
# 3. 결정변수
# ----------------------
# x^c[i,t,m]: 선박 i가 시간 t에 크레인 m을 계약시간으로 할당 (0/1)
# x^o[i,t,m]: 선박 i가 시간 t에 크레인 m을 초과근무로 할당 (0/1)
# r[i,t,m]: 할당 여부, r = x^c + x^o
# s[i,t,m]: 시간 t에 작업 시작 이벤트 (0→1 전이) (0/1)
# e[i,t,m]: 시간 t에 작업 종료 이벤트 (1→0 전이) (0/1)
# z[i,t,m]: 작업 시작 시점에서 종료 이벤트까지의 시간 간격 제한에 사용 (연속 변수)

x_c = {}
x_o = {}
r_var = {}
s_var = {}
e_var = {}
z = {}

# 선박 i가 사용할 수 있는 크레인 m은 available_cranes[i]를 기준으로 생성합니다.
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in available_cranes[i]:
            x_c[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")
            r_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"r_{i}_{t}_{m}")
            s_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"s_{i}_{t}_{m}")
            e_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"e_{i}_{t}_{m}")
            z[i,t,m]   = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"z_{i}_{t}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족 (입항 시각부터 출항 직전까지)
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m]
                 for t in range(b[i], d[i])
                 for m in available_cranes[i])
        == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인 (선박별)
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in available_cranes[i])
            >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in available_cranes[i])
            <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당 (전체 크레인 집합에서 검사)
# 단, 선박 i가 사용 가능한 크레인 외의 m은 모델에 변수가 생성되지 않으므로, 
# 모든 선박에 대해 m in {1,2,...,Q_total}으로 반복합니다.
for t in range(1, T+1):
    for m in range(1, Q_total+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1)
                     if m in available_cranes.get(i, []))
            <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )

# 4.5. 계약시간/초과 근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i])
                 for m in available_cranes[i])
        == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i])
                 for m in available_cranes[i])
        == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# 4.6. r 변수와의 연결: r = x_c + x_o
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in available_cranes[i]:
            model.addConstr(
                r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m],
                name=f"r_link_{i}_{t}_{m}"
            )

# 4.7. 연속 블록 제약 (시작/종료 이벤트)
# (A) 시간 전이 제약: t = b[i]+1, ..., d[i]-1
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                r_var[i,t,m] - r_var[i,t-1,m] == s_var[i,t,m] - e_var[i,t,m],
                name=f"cont_{i}_{t}_{m}"
            )
# (B) 초기 조건: t = b[i]
for i in range(1, N+1):
    for m in available_cranes[i]:
        t0 = b[i]
        model.addConstr(
            s_var[i,t0,m] == r_var[i,t0,m],
            name=f"s_init_{i}_{t0}_{m}"
        )
        model.addConstr(
            e_var[i,t0,m] == 0,
            name=f"e_init_{i}_{t0}_{m}"
        )
# (C) 단일 시작/종료: 각 선박·크레인에 대해 (전체 작업기간 동안 시작과 종료는 각각 한 번 발생)
for i in range(1, N+1):
    for m in available_cranes[i]:
        model.addConstr(
            quicksum(s_var[i,t,m] for t in range(b[i], d[i])) == 1,
            name=f"one_start_{i}_{m}"
        )
        model.addConstr(
            quicksum(e_var[i,t,m] for t in range(b[i], d[i])) == 1,
            name=f"one_end_{i}_{m}"
        )
# (D) 이벤트 발생 보완: t = b[i]+1, ..., d[i]-1
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                s_var[i,t,m] <= 1 - r_var[i,t-1,m],
                name=f"s_bound_{i}_{t}_{m}"
            )
            model.addConstr(
                e_var[i,t,m] <= r_var[i,t-1,m],
                name=f"e_bound_{i}_{t}_{m}"
            )

# 4.8. 크레인 연속성 제약 (같은 시간 슬롯에서 할당된 크레인 인덱스는 연속되어야 함)
# 만약 선박 i가 시간 t에 크레인 m와 n (m<n)을 할당받았다면, 그 사이의 모든 크레인 k (m < k < n)도 할당되어야 함.
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        # available_cranes[i]에 대해 오름차순 정렬된 리스트를 사용
        cranes = sorted(available_cranes[i])
        for idx_m in range(len(cranes) - 1):
            for idx_n in range(idx_m+1, len(cranes)):
                m_val = cranes[idx_m]
                n_val = cranes[idx_n]
                # m와 n 사이에 속하는 크레인들에 대해 제약 추가
                for m_mid in [cr for cr in cranes if m_val < cr < n_val]:
                    model.addConstr(
                        r_var[i,t,m_val] + r_var[i,t,n_val] - 1 <= r_var[i,t,m_mid],
                        name=f"contiguous_{i}_{t}_{m_val}_{n_val}_{m_mid}"
                    )

# 4.9. 추가 제약: z 제한 – t에서 시작한 작업은, 종료 이벤트가 발생한 t'까지의 간격을 초과할 수 없음.
# t = b[i] ... d[i]-1, t' = t+1 ... d[i]
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(b[i], d[i]):
            for tp in range(t+1, d[i]+1):
                model.addConstr(
                    z[i,t,m] <= (tp - t) + M1*(1 - e_var[i,tp,m]),
                    name=f"z_limit_{i}_{t}_{tp}_{m}"
                )

# 4.10. z 관련 제약:
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(b[i], d[i]):
            model.addConstr(
                z[i,t,m] <= M * s_var[i,t,m],
                name=f"z_le_Ms_{i}_{t}_{m}"
            )
            model.addConstr(
                z[i,t,m] <= r_var[i,t,m],
                name=f"z_bound_{i}_{t}_{m}"
            )
            model.addConstr(
                z[i,t,m] <= H,
                name=f"z_le_H_{i}_{t}_{m}"
            )

model.update()

# ----------------------
# 5. 목적함수
# ----------------------
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in available_cranes.get(i, [])
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력 (모든 t에 대해)
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}\n")
    print("시간 슬롯별 할증 계수:")
    for t in range(1, T+1):
        print(f"  t={t}: w = {w[t]}")
    print("\n각 선박별, 크레인별 변수 값 (모든 t):")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in available_cranes[i]:
            print(f"  크레인 {m}:")
            for t in range(1, T+1):
                print(f"    t={t}: r = {r_var[i,t,m].X:.0f}, x^c = {x_c[i,t,m].X:.0f}, x^o = {x_o[i,t,m].X:.0f}, s = {s_var[i,t,m].X:.0f}, e = {e_var[i,t,m].X:.0f}, z = {z[i,t,m].X:.0f}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1523 rows, 1584 columns and 3387 nonzeros
Model fingerprint: 0xa5bef3a0
Variable types: 264 continuous, 1320 integer (1320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 1452 rows and 1503 columns
Presolve time: 0.01s
Presolved: 71 rows, 81 columns, 258 nonzeros
Variable types: 0 continuous, 81 integer (81 binary)
Found heuristic solution: objective 260.0000000

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       260.00000  260.00000  0.00%     

In [2]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 4        # 선박 수
T = 24       # 전체 시간 슬롯 (1시간 단위: t=1,...,24)
Q_total = 10 # 전체 크레인 수

# 선박별 총 작업시간 (크레인-시간)
a = {1: 5, 2: 5, 3: 5, 4: 4}

# 입항 및 출항 시각 (t 단위)
b = {1: 5, 2: 3, 3: 11, 4: 15}
d = {1: 8, 2: 10, 3: 15, 4: 22}

# 선박별 사용 가능한 크레인 집합
available_cranes = {
    1: [1, 2, 3],
    2: [6, 7, 8],
    3: [1, 2, 3],
    4: [4, 5]
}

# 시간당 최소/최대 크레인 할당
qmin = {1: 0, 2: 0, 3: 0, 4: 0}
qmax = {i: len(available_cranes[i]) for i in available_cranes}

# 계약시간 (최대 계약시간)
H = 3

# 단가
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가

# 야간 할증 계수
w = {t: 1.5 if (t >= 18 or t <= 4) else 1.0 for t in range(1, T+1)}

# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP_NoBigM")

# ----------------------
# 3. 결정변수
# ----------------------
x_c = {}  # 계약시간 할당 (0/1)
x_o = {}  # 초과근무 할당 (0/1)
r_var = {}  # 할당 여부 (r = x^c + x^o)
Ts = {}  # 작업 시작 시간 (정수)
Te = {}  # 작업 종료 시간 (정수)

for i in range(1, N+1):
    for t in range(1, T+1):
        for m in available_cranes[i]:
            x_c[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")
            r_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"r_{i}_{t}_{m}")
    for m in available_cranes[i]:
        Ts[i,m] = model.addVar(lb=b[i], ub=d[i]-1, vtype=GRB.INTEGER, name=f"Ts_{i}_{m}")
        Te[i,m] = model.addVar(lb=b[i]+1, ub=d[i], vtype=GRB.INTEGER, name=f"Te_{i}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in available_cranes[i]:
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m] for t in range(b[i], d[i]) for m in available_cranes[i]) == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in available_cranes[i]) >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in available_cranes[i]) <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당
for t in range(1, T+1):
    for m in range(1, Q_total+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1) if m in available_cranes.get(i, [])) <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )

# 4.5. 계약시간/초과 근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i]) for m in available_cranes[i]) == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i]) for m in available_cranes[i]) == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# 4.6. r 변수와의 연결
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in available_cranes[i]:
            model.addConstr(
                r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m],
                name=f"r_link_{i}_{t}_{m}"
            )

# 4.7. 작업 시작/종료 시간과 r 변수 연결
for i in range(1, N+1):
    for m in available_cranes[i]:
        # Ts <= Te
        model.addConstr(Ts[i,m] <= Te[i,m], name=f"Ts_le_Te_{i}_{m}")
        # r[i,t,m]이 Ts와 Te 사이에서만 1이 되도록 제약
        for t in range(b[i], d[i]):
            # t < Ts[i,m]이면 r[i,t,m] = 0
            model.addConstr(r_var[i,t,m] * (Ts[i,m] - t) <= 0, name=f"r_after_Ts_{i}_{t}_{m}")
            # t >= Te[i,m]이면 r[i,t,m] = 0
            model.addConstr(r_var[i,t,m] * (t - Te[i,m]) <= 0, name=f"r_before_Te_{i}_{t}_{m}")
        # 작업량과 Ts, Te 연결
        model.addConstr(
            quicksum(r_var[i,t,m] for t in range(b[i], d[i])) == Te[i,m] - Ts[i,m],
            name=f"duration_{i}_{m}"
        )

# 4.8. 크레인 연속성 제약
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        cranes = sorted(available_cranes[i])
        for idx_m in range(len(cranes) - 1):
            for idx_n in range(idx_m+1, len(cranes)):
                m_val = cranes[idx_m]
                n_val = cranes[idx_n]
                for m_mid in [cr for cr in cranes if m_val < cr < n_val]:
                    model.addConstr(
                        r_var[i,t,m_val] + r_var[i,t,n_val] - 1 <= r_var[i,t,m_mid],
                        name=f"contiguous_{i}_{t}_{m_val}_{n_val}_{m_mid}"
                    )

# ----------------------
# 5. 목적함수
# ----------------------
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in available_cranes.get(i, [])
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}\n")
    print("시간 슬롯별 할증 계수:")
    for t in range(1, T+1):
        print(f"  t={t}: w = {w[t]}")
    print("\n각 선박별, 크레인별 변수 값:")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in available_cranes[i]:
            print(f"  크레인 {m}: Ts = {Ts[i,m].X}, Te = {Te[i,m].X}")
            for t in range(b[i], d[i]):
                print(f"    t={t}: r = {r_var[i,t,m].X:.0f}, x^c = {x_c[i,t,m].X:.0f}, x^o = {x_o[i,t,m].X:.0f}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1010 rows, 814 columns and 2326 nonzeros
Model fingerprint: 0xe3e036e5
Model has 112 quadratic constraints
Variable types: 0 continuous, 814 integer (792 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e+00, 2e+01]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 5e+00]
Presolve removed 947 rows and 748 columns
Presolve time: 0.03s
Presolved: 63 rows, 66 columns, 219 nonzeros
Variable types: 0 continuous, 66 integer (60 binary)
Found heuristic solution: objective 260.0000000

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   